# LAMMPS Parser Example Notebook

This notebook demonstrates how to use the `lammps_parser` package to:
- **Sanitize** a LAMMPS script (remove comments, expand loops, resolve variables).
- **Parse** the cleaned script into an Abstract Syntax Tree (AST) using Lark.
- **Visualize** the parsed structure.

## Installation
Before using this package, make sure it's installed.

In [ ]:
import os
from lark.tree import pydot__tree_to_png
from IPython.display import Image, display

from lammps_ast.parser import parse_to_AST
from lammps_ast.sanitizer import sanitize
from lammps_ast.comparator import highlight_tree_differences

## Example Script

In [ ]:
sample_script_content = sample_script_content = """  
# LAMMPS Simulation Input - Needs Sanitization

units           metal
dimension       3
boundary        p p p
atom_style      atomic

variable       temp equal 300
variable       density equal 8.9
variable       scale_factor equal ${temp} * 0.5  # Potential issue: temp is undefined at this point

lattice        fcc 3.615
region         box block 0 10 0 10 0 10
create_box     1 box

create_atoms   1 box

mass           1 58.6934  # Nickel
velocity       all create ${temp} 12345 mom yes rot yes dist gaussian

# Minimize energy (indentation issue)
      minimize 1.0e-4 1.0e-6 1000 10000    

fix            1 all nvt temp ${temp} ${temp} 0.1

thermo         100
dump           1 all custom 100 dump.lammpstrj id type x y z
run            5000
"""

# Save this script to a temporary file
sample_filename = "sample_lammps_script.lammps"

with open(sample_filename, "w") as file:
    file.write(sample_script_content)

print(f"Sample script saved as {sample_filename}")


## Sanitizing

In [ ]:
with open(sample_filename, "r") as file:
    original_script = file.read()

cleaned_script = sanitize(original_script)

print("Sanitized Script:\n")
print(cleaned_script)

## Parsing

In [ ]:
ast = parse_to_AST(sample_filename)

if ast:
    print("\nParsed AST:\n")
    print(ast.pretty())
else:
    print("Parsing failed.")

## Visualizing

In [ ]:
if ast:
    ast_image_path = "ast_visualization.png"
    pydot__tree_to_png(ast, ast_image_path)
    display(Image(ast_image_path))

## Comparing

In [ ]:
# Sample LAMMPS scripts to compare
script1 = """  
units           metal
atom_style      atomic

variable        temp equal 300

velocity        all create ${temp} 12345 mom yes rot yes

fix            1 all nvt temp ${temp} ${temp} 0.1

thermo         100
run            5000
"""

with open("sample_lammps_script1.lammps", "w") as file:
    file.write(script1)

script2 = """  
units           metal
atom_style      atomic

variable        temp equal 500  # Different initial temperature
velocity        all create ${temp} 54321 mom yes rot yes

fix            1 all nve  # Different fix: NVE instead of NVT

thermo         50  # More frequent thermo output
run            10000  # Longer simulation
"""
with open("sample_lammps_script2.lammps", "w") as file:
    file.write(script2)

In [ ]:

# Parse both scripts into ASTs
tree1 = parse_to_AST("sample_lammps_script1.lammps")
tree2 = parse_to_AST("sample_lammps_script2.lammps")

# Compute and display differences
if tree1 and tree2:
    print("\n🔍 Tree Comparison Results:")
    highlight_tree_differences(tree1, tree2)
